In [7]:
import os
import logging
import importlib
import framework
importlib.reload(framework)
import test_classifier
importlib.reload(test_classifier)
import data_utils
importlib.reload(data_utils)
from framework import Usecase, DataCuration, FeatureEngineering, ModelEvaluator
import pandas as pd
# Define some constants and configurations
logging.getLogger().setLevel(logging.INFO)

ACCESS_TOKEN = 'WUpGevbWC9lsnTW8quNUtmWRdAEM89'

Set up the usecase details

In [8]:
USECASE = 'ner'
USECASE_CONFIG = {
    'dataset': 'w2',
    'num_labels': 3,
    'labels_dict': {'person' : 0, 'org' : 1, 'address' : 2, 'none': 3}
}

usecase = Usecase(USECASE)
usecase.setConfig(USECASE_CONFIG)

Set paths for datasets and goldens (local or ib, both work).
Specify configurations

In [9]:
DATAS = {
    ('ner', 'w2') : '/Users/ahsaasbajaj/Documents/Data/w2-instabase/flow/s2_map_records'
}

GOLDENS = {
  ('ner', 'w2') : '/Users/ahsaasbajaj/Documents/Data/w2-instabase/golden/goldens.csv'
}

DATASET = USECASE_CONFIG['dataset']
DATA = DATAS[(USECASE, DATASET)]
GOLDEN = GOLDENS[(USECASE, DATASET)]

GOLDEN_CONFIG = {
    'is_local': True,
    'index_field_name':'filename',
    'file_type': 'csv',
    'skip_first_row': True,
    'identifier': 'file'
}
DATASET_CONFIG = {
    'is_local': True, 
    'file_type': 'ibocr',
    'identifier': lambda path: os.path.basename(path).split('.ibocr')[0],
    'convert2txt': False
}

w2 = DataCuration(ACCESS_TOKEN, DATA, DATASET_CONFIG, GOLDEN, GOLDEN_CONFIG)   

INFO:root:Loading dataset from /Users/ahsaasbajaj/Documents/Data/w2-instabase/flow/s2_map_records
INFO:root:Loading goldens from /Users/ahsaasbajaj/Documents/Data/w2-instabase/golden/goldens.csv
Total files Goldens:  (154, 25)
Total files found in the source (142, 25)


In [10]:
PROCESSING_CONFIG = {
    'X_DIST_THRESHOLD': 100
}
CANDIDATES_FIELDS = {
    'person':'employee_name',
    'org':'employer_name'
}
w2.processIBOCR2candidatePhrases(DATASET_CONFIG, PROCESSING_CONFIG)
w2.compare_candidates_and_goldens(PROCESSING_CONFIG, CANDIDATES_FIELDS)

Generating candidates for 142 files
For X_DIST_THRESHOLD configuraion: 100
total files: 142
person names found in candidates: 131
org names found in candidates: 79



Generate test data from goldens (from actual persons and company names)

In [11]:
fe = FeatureEngineering(usecase, w2, CANDIDATES_FIELDS)
test_data = fe.generate_test_samples_from_goldens()
test_data.head()

,context,label
45,MARY BORO,0
233,AMAZON FRESH LLC,1
150,JPMORGAN CHASE BANK,1
171,QUICKEN LOANS INC.,1
124,MELANIE T BAUTISTA,0


Load trained (fine-tuned models) for inference

In [6]:
MODEL_TRAIN_DATA = 'w2'

MODELS = {
    ('ner', 'w2') : '/Users/ahsaasbajaj/Documents/Code/ner-hf/sequence-classification/w2/no-address/5/model.pt',
    ('ner', 'public') : '/Users/ahsaasbajaj/Documents/Code/ner-hf/sequence-classification/public/no-address/200/model.pt'
}

MODEL_CONFIG = {
    'model_file_or_path' : MODELS[(USECASE, MODEL_TRAIN_DATA)],
    'num_labels': USECASE_CONFIG['num_labels'],
    'gpu': False
}

model_evaluator = ModelEvaluator()
model_evaluator.setModelConfig(MODEL_CONFIG)
output = model_evaluator.runEvaluation(test_data)

NameError: name 'test_data' is not defined